# Malnet-Graph-tiny

NOTE：Run this notebook in SageMaker instance instead of SageMaker Studio

paper: https://mal-net.org/ https://mal-net.org/#Citation

code: https://github.com/safreita1/malnet-graph

datasets: http://malnet.cc.gatech.edu/graph-data/


Core requirements:
1. PyTorch1.13.1+py39
2. extended pre-built container + BYOC
3. SageMaker+Malnet-Graph
4. BYOS 
5. using FastFile mode to access dataset files in S3

Steps:
1. Generate Dockerfile
2. Generate requirements.txt
3. Build docker image
4. SageMaker setting
5. Download datasets and upload to s3
6. Upload container image to Amazon ECR
7. Start SageMaker training job

## 1. Generate Dockerfile

In [1]:
%%writefile Dockerfile
# SageMaker PyTorch image
FROM    763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker

ENV PATH="/opt/ml/code:${PATH}"

# this environment variable is used by the SageMaker PyTorch container to determine our user code directory.
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code


COPY requirements.txt /tmp/
RUN pip install --requirement /tmp/requirements.txt

WORKDIR /opt/ml/code

Overwriting Dockerfile


## 2. Generate requirements.txt

In [2]:
%%writefile requirements.txt
torch-geometric==2.0.3
torch-scatter
torch-sparse
tensorboard
networkx
grakel
gensim
karateclub

Overwriting requirements.txt


## 3. Build docker image #approximately 30 minutes first time

In [3]:
%%sh
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com
algorithm_name='malnet-graph'

docker build -t ${algorithm_name} .

Login Succeeded

Step 1/6 : FROM    763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker
 ---> f0cd3f7ded0e
Step 2/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 18681f60e144
Step 3/6 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> bedb4b0b014a
Step 4/6 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> 7f0eb3fdfd3f
Step 5/6 : RUN pip install --requirement /tmp/requirements.txt
 ---> Using cache
 ---> b70b2a055e9e
Step 6/6 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 4a09fe88c186
Successfully built 4a09fe88c186
Successfully tagged malnet-graph:latest


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## 4. SageMaker setting

In [4]:
%%time
#! python3 -m pip install --upgrade sagemaker
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput


sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
algorithm_name='malnet-graph'
prefix = "malnet-graph"
dataset_path="datasets/malnet-graph-tiny/"

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
role_name = role.split(["/"][-1])
print(f"SageMaker Execution Role: {role}")
print(f"The name of the Execution role: {role_name[-1]}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

SageMaker Execution Role: arn:aws:iam::432088571089:role/AmazonSageMaker-ExecutionRole-20210324T123126
The name of the Execution role: AmazonSageMaker-ExecutionRole-20210324T123126
AWS account: 432088571089
AWS region: us-east-1
CPU times: user 1.04 s, sys: 246 ms, total: 1.28 s
Wall time: 1.24 s


## 5. download datasets and upload to s3

In [5]:
# malnet-graphs-tiny.tar.gz
!rm -fr malnet-graphs-tiny.tar.gz malnet-graphs-tiny
!wget http://malnet.cc.gatech.edu/graph-data/malnet-graphs-tiny.tar.gz
!tar zxvf malnet-graphs-tiny.tar.gz
!aws s3 sync malnet-graphs-tiny s3://{bucket}/{dataset_path}malnet-graphs-tiny
#!rm -fr malnet-graphs-tiny.tar.gz malnet-graphs-tiny

--2023-03-06 03:49:53--  http://malnet.cc.gatech.edu/graph-data/malnet-graphs-tiny.tar.gz
Resolving malnet.cc.gatech.edu (malnet.cc.gatech.edu)... 130.207.126.102
Connecting to malnet.cc.gatech.edu (malnet.cc.gatech.edu)|130.207.126.102|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42204344 (40M) [application/octet-stream]
Saving to: ‘malnet-graphs-tiny.tar.gz’

100%[======================================>] 42,204,344  26.8MB/s   in 1.5s   

2023-03-06 03:49:54 (26.8 MB/s) - ‘malnet-graphs-tiny.tar.gz’ saved [42204344/42204344]

malnet-graphs-tiny/
malnet-graphs-tiny/addisplay/
malnet-graphs-tiny/addisplay/kuguo/
malnet-graphs-tiny/addisplay/kuguo/76EBFC2198D25806E336531F77F411BA76AAC1DB2ED11425B2B2C9443EC8E0DB.edgelist
malnet-graphs-tiny/addisplay/kuguo/5702368F8924F101F39C60F65B5BB5F90797F9EF05FC721DABFC33AFB4F46AA1.edgelist
malnet-graphs-tiny/addisplay/kuguo/75E08E4DDE6A9AF2982ACEBF9BDFDB45472559D68DA35D2A0AB2681A6D3D30A4.edgelist
malnet-graphs-tiny/addisp

malnet-graphs-tiny/addisplay/kuguo/64D71320088547378673FE11131B67F330434E27868A895941E387DD9BD662D9.edgelist
malnet-graphs-tiny/addisplay/kuguo/247FA8EED7BB046884F1D0CFBFE571AC421140AB3C5D0E4FE3CD6BED1C29FD6F.edgelist
malnet-graphs-tiny/addisplay/kuguo/ECDF113D57DEFAC9C4082725F3335A21774B81F909BD7B8F38466C153C7EC3BF.edgelist
malnet-graphs-tiny/addisplay/kuguo/66F94EE56CD8F9D2525A696E262FEBFCEDF582102DFB4F10ABDB8F90C0C23BB2.edgelist
malnet-graphs-tiny/addisplay/kuguo/51D9DDB05F4055F697831E8F0C03498D38C4433FB9AEDC7D66F708416DEBA33C.edgelist
malnet-graphs-tiny/addisplay/kuguo/BF56028ED99D1E890BE022F075B3E59366CC330B57E6B06C8B2E51CC7C4B18C7.edgelist
malnet-graphs-tiny/addisplay/kuguo/7D89E0D3A20943305EF1E78EE20AF894ACE0F841F0A9B84C94A0DA34B130228F.edgelist
malnet-graphs-tiny/addisplay/kuguo/59546114DAC67D61BC0B46CCB3EB6D41E4A3526B49E303AAABDEAFC2DD224491.edgelist
malnet-graphs-tiny/addisplay/kuguo/85D5F2A2EDC08AC94332B0EACC83C0AFD4A81B1D2851761FC6FC12ED27B58C6A.edgelist
malnet-graphs-tiny/

malnet-graphs-tiny/addisplay/kuguo/40969A8B0A62AF9E56DD55EE69D2EBF43F44A6EBD11A5BF31655E9C6C60648C6.edgelist
malnet-graphs-tiny/addisplay/kuguo/A95F0260D6D90306EB506890F6DBC8B613DA249C2892B23A6D698132D9D6661E.edgelist
malnet-graphs-tiny/addisplay/kuguo/78E2FFDE0BE16AC93A49DA9116F65007B8E028B53C8F62AC71292031DC060B80.edgelist
malnet-graphs-tiny/addisplay/kuguo/858F2E555457711052A6161216A453C6DB63391C58B95F583472B7948BC0B596.edgelist
malnet-graphs-tiny/addisplay/kuguo/CCAC75D79157412802BAB5D88F06DE4C30CD543263B3FFE4EBD86B261595D63A.edgelist
malnet-graphs-tiny/addisplay/kuguo/344670E8636D873F5D6BEC5BA6316B86DA7343A4E3B71C7448B797F6EF60AD77.edgelist
malnet-graphs-tiny/addisplay/kuguo/902C062F5C2635C195C269AE496FB4E7AB83486F656D4D4ED051A25EBDAFDCA6.edgelist
malnet-graphs-tiny/addisplay/kuguo/010A0663BEA9B58245997646F71F67168DCFE9EDEACA0665AA5894EFB9E9932D.edgelist
malnet-graphs-tiny/addisplay/kuguo/20431A3294A24BAB350C35335FCEFDA43E320BE8951E645A9A509CCE7364410F.edgelist
malnet-graphs-tiny/

malnet-graphs-tiny/addisplay/kuguo/9EDA943C63611D7540EE82C6AAC089BA65D288758DD40E5C88A6EAEAD075EBD0.edgelist
malnet-graphs-tiny/addisplay/kuguo/9575A6F8E4D739B1E6F169243D467BCA36F126F3EB39656E8296F86BDE58E881.edgelist
malnet-graphs-tiny/addisplay/kuguo/0850231C5AF67174F7CE654BD08928C1A4223955C543C0F2CB64857593CA928F.edgelist
malnet-graphs-tiny/addisplay/kuguo/2B37C818D426E2808D6BFEC277A64D2EA4B5F6A1AAF97C90DF57DEA4CB2F0818.edgelist
malnet-graphs-tiny/addisplay/kuguo/2944C4E1070B722476DA2F2B90A51966625561511BF7EC157C6403B896F071FD.edgelist
malnet-graphs-tiny/addisplay/kuguo/C17EF48E924743E6135A53DD2845F242931A6F60D3723D026E505B4B27E46FEA.edgelist
malnet-graphs-tiny/addisplay/kuguo/251B52753C0CD24D4333C08725FBF7B4C5FFCBD2C0A45412E33E7515D5799D73.edgelist
malnet-graphs-tiny/addisplay/kuguo/1F7A609F5F366339320A6C226C6E19BE1BA836240BF395E395FA7ACE5199E7A1.edgelist
malnet-graphs-tiny/addisplay/kuguo/8C524EE2FD40455FA32B7BB2A922EBB18BFA2BF9B93A800B02C674A2323D9910.edgelist
malnet-graphs-tiny/

malnet-graphs-tiny/trojan/artemis/39DE15B9F9C64A50F1E23FF352C4A34571DDAD304147DF502B6263A15F047540.edgelist
malnet-graphs-tiny/trojan/artemis/227F4D216D9F7721765097A740E18727915D269ABBA4B6D0EC3962F668F0FDEA.edgelist
malnet-graphs-tiny/trojan/artemis/1A20C1041F5BE032EAB31667A808EDFB4EB0FD0C4E72D7A6EED592743C346780.edgelist
malnet-graphs-tiny/trojan/artemis/A2966BF77D8CB077BC9BEC688234E8243C6ED4DD227224509429CFA96AEBBE5B.edgelist
malnet-graphs-tiny/trojan/artemis/9EC71EC4EEEBE2DB18E8B5BEDF61F193E43C5C8294791EA5D2856EE7859ED0CB.edgelist
malnet-graphs-tiny/trojan/artemis/4E90A6E960048BC13230B2008BFD1B1014AB82C533FA66C12904F25D80AED757.edgelist
malnet-graphs-tiny/trojan/artemis/AAED5D8D4664B417D5474CF05E9F4640297A232C9015DF3E231E2EA00456DB3D.edgelist
malnet-graphs-tiny/trojan/artemis/66D8B5EA19FDD43D36786DBB4348EF277B9C2AD8DEF1009A4CC3D0D95B7E26D3.edgelist
malnet-graphs-tiny/trojan/artemis/CD9EAB9B4796A608AFBF861D7CFC36E7C1089B5F17310B418CCD84098209CA0E.edgelist
malnet-graphs-tiny/trojan/ar

...

upload: malnet-graphs-tiny/trojan/artemis/E4E39CECFA5F1903408BD156C46E8A415904ED6A27C65FBD5F9C3EB83705F1F9.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/E4E39CECFA5F1903408BD156C46E8A415904ED6A27C65FBD5F9C3EB83705F1F9.edgelist
upload: malnet-graphs-tiny/trojan/artemis/E4BF4EE27ABC495D2DFE68EE32FCC334DC46C43B51E7319D0CAD1A45BDDBD928.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/E4BF4EE27ABC495D2DFE68EE32FCC334DC46C43B51E7319D0CAD1A45BDDBD928.edgelist
upload: malnet-graphs-tiny/trojan/artemis/E67C8799A7E01CC8085C29E6AFAD779A081B242D398D493FB84B2909B8DE4247.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/E67C8799A7E01CC8085C29E6AFAD779A081B242D398D493FB84B2909B8DE4247.edgelist
upload: malnet-graphs-tiny/trojan/artemis/E68DDC955C59708F73AB383DF885443838ABA758E4F33620B494844C5BCD0417.edgelist to s3://

upload: malnet-graphs-tiny/trojan/artemis/E9FC1195EC2E6611721ACC70DA3F71FAD371574B8057436E50080E1ADA66971B.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/E9FC1195EC2E6611721ACC70DA3F71FAD371574B8057436E50080E1ADA66971B.edgelist
upload: malnet-graphs-tiny/trojan/artemis/EBDAE9152D12F4D7BCAD647075AC6854838E390F716DF40E3CBE4A25D8340B5D.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/EBDAE9152D12F4D7BCAD647075AC6854838E390F716DF40E3CBE4A25D8340B5D.edgelist
upload: malnet-graphs-tiny/trojan/artemis/EC53517345324D4FB39BEB4E50465D31EF5FB23D18E57019B6FBD3322EAB07E7.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/EC53517345324D4FB39BEB4E50465D31EF5FB23D18E57019B6FBD3322EAB07E7.edgelist
upload: malnet-graphs-tiny/trojan/artemis/EB0F7007368F7E9AABFFB14B3373B40F59834B86F1962C2BE65FAE4F66777A4F.edgelist to s3://

upload: malnet-graphs-tiny/trojan/artemis/EF7E89EC74506470FB1EE141F46B667A21A7BF53F5C2FFD0545E4C9FF5873E06.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/EF7E89EC74506470FB1EE141F46B667A21A7BF53F5C2FFD0545E4C9FF5873E06.edgelist
upload: malnet-graphs-tiny/trojan/artemis/F00E8F42461ADC63D518FF9A7CF6E831E63F5022E4608E995B778F9AC400D9F0.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/F00E8F42461ADC63D518FF9A7CF6E831E63F5022E4608E995B778F9AC400D9F0.edgelist
upload: malnet-graphs-tiny/trojan/artemis/F1EDD820111AF61A3B4F1A5A9ED429CFAD24166B7B763C3E2124D96676D0BCF2.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/F1EDD820111AF61A3B4F1A5A9ED429CFAD24166B7B763C3E2124D96676D0BCF2.edgelist
upload: malnet-graphs-tiny/trojan/artemis/F087C0FD7C0C4B40BC91D9B191D547DA758FB0AE7E9C4882728DCAB333255FB1.edgelist to s3://

upload: malnet-graphs-tiny/trojan/artemis/F917E0BB2DF315D1DC04F989A6449C7E6F43F3F4D7B0ED19134DECA4A2CFA205.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/F917E0BB2DF315D1DC04F989A6449C7E6F43F3F4D7B0ED19134DECA4A2CFA205.edgelist
upload: malnet-graphs-tiny/trojan/artemis/F48E81FD0E2DC9D786640A3FB39A0770F93B107C459C57986FABF536F6D510E0.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/F48E81FD0E2DC9D786640A3FB39A0770F93B107C459C57986FABF536F6D510E0.edgelist
upload: malnet-graphs-tiny/trojan/artemis/F744A0FB17A81944DF37C25082A082AC8D7737A2E2BCB601C4DC857C230E68D4.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/F744A0FB17A81944DF37C25082A082AC8D7737A2E2BCB601C4DC857C230E68D4.edgelist
upload: malnet-graphs-tiny/trojan/artemis/FA8035224DB051FF7661C3F890AC560171BB6D2D609DE8CC0CA74C6AA7C1FAA9.edgelist to s3://

upload: malnet-graphs-tiny/trojan/artemis/FFCB9454237577997C3CE349B98D439F3DECC8088E2E4050CA971A46529C1577.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/FFCB9454237577997C3CE349B98D439F3DECC8088E2E4050CA971A46529C1577.edgelist
upload: malnet-graphs-tiny/trojan/artemis/FE41B6655E183875F4EEE7692CC4A292237ED9CFA80FD1E3484222A547193C55.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/FE41B6655E183875F4EEE7692CC4A292237ED9CFA80FD1E3484222A547193C55.edgelist
upload: malnet-graphs-tiny/trojan/artemis/FEBD228E1AB8CA40B923678C98097A98D86F13929DD03B1BBA36FC5CC77CC255.edgelist to s3://sagemaker-us-east-1-432088571089/datasets/malnet-graph-tiny/malnet-graphs-tiny/trojan/artemis/FEBD228E1AB8CA40B923678C98097A98D86F13929DD03B1BBA36FC5CC77CC255.edgelist


In [6]:
#split_info_tiny
!wget https://github.com/safreita1/malnet-graph/blob/master/split_info_tiny/type/test.txt?raw=True -O test.txt
!wget https://github.com/safreita1/malnet-graph/blob/master/split_info_tiny/type/train.txt?raw=True -O train.txt
!wget https://github.com/safreita1/malnet-graph/blob/master/split_info_tiny/type/val.txt?raw=True -O val.txt

!aws s3 cp test.txt s3://{bucket}/{dataset_path}split_info_tiny/type/1.0/test.txt
!aws s3 cp train.txt s3://{bucket}/{dataset_path}split_info_tiny/type/1.0/train.txt
!aws s3 cp val.txt s3://{bucket}/{dataset_path}split_info_tiny/type/1.0/val.txt
    
#!rm -fr test.txt train.txt val.txt

--2023-03-06 03:50:24--  https://github.com/safreita1/malnet-graph/blob/master/split_info_tiny/type/test.txt?raw=True
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/safreita1/malnet-graph/raw/master/split_info_tiny/type/test.txt [following]
--2023-03-06 03:50:24--  https://github.com/safreita1/malnet-graph/raw/master/split_info_tiny/type/test.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/safreita1/malnet-graph/master/split_info_tiny/type/test.txt [following]
--2023-03-06 03:50:24--  https://raw.githubusercontent.com/safreita1/malnet-graph/master/split_info_tiny/type/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.

In [7]:
#!aws s3 ls s3://{bucket}

## 6. Upload container image to Amazon ECR

In [8]:
%%sh
algorithm_name='malnet-graph'
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}



Login Succeeded

Step 1/6 : FROM    763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker
 ---> f0cd3f7ded0e
Step 2/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 18681f60e144
Step 3/6 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> bedb4b0b014a
Step 4/6 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> 7f0eb3fdfd3f
Step 5/6 : RUN pip install --requirement /tmp/requirements.txt
 ---> Using cache
 ---> b70b2a055e9e
Step 6/6 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 4a09fe88c186
Successfully built 4a09fe88c186
Successfully tagged malnet-graph:latest
The push refers to repository [432088571089.dkr.ecr.us-east-1.amazonaws.com/malnet-graph]
b22e4697d124: Preparing
57c8b4cd9216: Preparing
b1cb7b67ba39: Preparing
f5f76489fff8: Preparing
621c3f07daa7: Preparing
9b484bb42e11: Preparing
54c7c0b58471: Preparing
c34adc3ab668: Preparing
bbf651e48b84: Preparing
f61045791108: Preparing
4e2ac0cda74a: Prepar

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



# 7. Start SageMaker training job

### 7.1 dm_experiments  #approximately  3219 seconds

In [9]:
%%time
import time
from time import gmtime, strftime
# please modify this byoc_image_uri variable
byoc_image_uri=f'{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest'
train_channel=TrainingInput(
        s3_data=f's3://{bucket}/{dataset_path}',
        input_mode='FastFile'  # Available options: File | Pipe | FastFile
    )

entry_point='dm_experiments.py'# gnn_experiments.py
source_dir='dm'# gnn

estimator = Estimator(image_uri=byoc_image_uri,
                      role=role,
                      entry_point=entry_point,
                      source_dir=source_dir,
                      instance_count=1,
                      instance_type='ml.g4dn.xlarge')

estimator.fit(inputs={"train":train_channel},
              job_name="malnet-graph-"+time.strftime("%Y%m%d%H%M%S", time.localtime()))

INFO:sagemaker:Creating training-job with name: malnet-graph-20230306035031


2023-03-06 03:50:31 Starting - Starting the training job...
2023-03-06 03:50:51 Starting - Preparing the instances for training......
2023-03-06 03:51:48 Downloading - Downloading input data...
2023-03-06 03:52:08 Training - Downloading the training image.....................
2023-03-06 03:55:59 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
2023-03-06 03:56:05,615 sagemaker-training-to

84%|████████▍ | 2936/3500 [00:54<00:10, 52.76it/s]


100%|██████████| 3500/3500 [01:03<00:00, 55.03it/s]
******Running vnge_slaq on val set******
100%|██████████| 500/500 [00:08<00:00, 61.29it/s]
******Running vnge_slaq on test set******
100%|██████████| 1000/1000 [00:17<00:00, 57.60it/s]
Classifying vnge_slaq graph embeddings...
100%|██████████| 16/16 [00:01<00:00,  8.93it/s]
Best val macro-F1: 0.5645302901474367
******Running vnge_slaq on train set******
25%|██▌       | 880/3500 [00:16<00:40, 64.86it/s]


90%|█████████ | 3152/3500 [01:02<00:05, 62.89it/s]


...

39%|███▉      | 1360/3500 [00:59<01:20, 26.60it/s]


77%|███████▋  | 2680/3500 [02:00<00:35, 23.16it/s]


100%|██████████| 3500/3500 [02:34<00:00, 22.67it/s]
******Running ldp on val set******
91%|█████████ | 456/500 [00:20<00:02, 20.89it/s]


100%|██████████| 500/500 [00:21<00:00, 22.89it/s]
******Running ldp on test set******
100%|██████████| 1000/1000 [00:45<00:00, 22.20it/s]
Classifying ldp graph embeddings...
100%|██████████| 16/16 [00:01<00:00,  8.24it/s]
Best val macro-F1: 0.8706220120046131
Method=vnge_slaq, malnet_tiny=True, group=type, train_ratio=1.0, n_iter=0, order=0, n_eigen=0, n_vectors=10, n_steps=10, val_macro-F1=0.5645302901474367, test_macro-F1=0.5424370023876284
Method=vnge_slaq, malnet_tiny=True, group=type, train_ratio=1.0, n_iter=0, order=0, n_eigen=0, n_vectors=15, n_steps=15, val_macro-F1=0.5541041345485791, test_macro-F1=0.5414057869519007
Method=vnge_slaq, malnet_tiny=True, group=type, train_ratio=1.0, n_iter=0, order=0, n_eigen=0, n_vectors=20, n_steps=20, val_macro-F1=0.567274379145122, test_macro-F1=0.5376502321254767
Method=lsd_slaq, malnet_tiny=True, group=type, train_ratio=1.0, n_iter=0, order=0, n_eigen=0, n_vectors=10, n_steps=10, val_macro-F1=0.7931804592727436, test_macro-F1=0.77745227462


2023-03-06 04:45:27 Uploading - Uploading generated training model
2023-03-06 04:45:27 Completed - Training job completed
Training seconds: 3219
Billable seconds: 3219
CPU times: user 14.2 s, sys: 257 ms, total: 14.5 s
Wall time: 55min 17s


In [10]:
print(estimator.model_data)

s3://sagemaker-us-east-1-432088571089/malnet-graph-20230306035031/output/model.tar.gz


### 7.2 gnn_experiments #approximately 8 minutes 22seconds per 50 epochs 

In [15]:
%%time
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
# please modify this byoc_image_uri variable
byoc_image_uri=f'{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest'
train_channel=TrainingInput(
        s3_data=f's3://{bucket}/{dataset_path}',
        input_mode='FastFile'  # Available options: File | Pipe | FastFile
    )

entry_point='gnn_experiments.py'# gnn_experiments.py
source_dir='gnn'# gnn

estimator = Estimator(image_uri=byoc_image_uri,
                      role=role,
                      entry_point=entry_point,
                      source_dir=source_dir,
                      instance_count=1,
                      instance_type='ml.g4dn.xlarge')

estimator.fit(inputs={"train":train_channel},
              job_name="malnet-graph-"+time.strftime("%Y%m%d%H%M%S", time.localtime()))

INFO:sagemaker:Creating training-job with name: malnet-graph-20230306050346


2023-03-06 05:03:47 Starting - Starting the training job...
2023-03-06 05:04:03 Starting - Preparing the instances for training......
2023-03-06 05:05:05 Downloading - Downloading input data...
2023-03-06 05:05:31 Training - Downloading the training image.....................
2023-03-06 05:08:57 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
2023-03-06 05:09:08,516 sagemaker-training-tool

8%|▊         | 276/3500 [00:08<00:46, 68.67it/s]
#033[A
14%|█▎        | 476/3500 [00:11<00:46, 65.15it/s]
#033[A
17%|█▋        | 604/3500 [00:13<00:54, 53.51it/s]
#033[A
23%|██▎       | 812/3500 [00:17<00:41, 65.36it/s]
#033[A
26%|██▌       | 908/3500 [00:18<00:35, 73.02it/s]
#033[A
53%|█████▎    | 1852/3500 [00:34<00:29, 56.64it/s]#033[A


75%|███████▌  | 2636/3500 [00:47<00:15, 56.34it/s]
#033[A
82%|████████▏ | 2860/3500 [00:51<00:10, 58.60it/s]
#033[A
59%|█████▉    | 296/500 [00:04<00:03, 64.06it/s]
#033[A
78%|███████▊  | 392/500 [00:06<00:01, 54.11it/s]#033[A


92%|█████████▏| 920/1000 [00:15<00:01, 56.70it/s]
#033[A
100%|██████████| 1000/1000 [00:16<00:00, 58.50it/s]#033[A#015100%|██████████| 1000/1000 [00:16<00:00, 59.50it/s]
/opt/conda/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
0%|          | 0/55 [00:00<?, ?it/s]#033[A
INFO:root:Using NamedTuple = typing._NamedTuple instead.
[2023-03-06 05:10:43.651 ip-10-2-194-134.ec2.internal:42 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-03-06 05:10:43.970 ip-10-2-194-134.ec2.internal:42 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-03-06 05:10:43.971 ip-10-2-194-134.ec2.internal:42 INFO json_config.py:92] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2023-03-06 05:10:43.971 ip-10-2-194-134.ec2.internal:42 INFO hook.py:206] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard sum

100%|██████████| 55/55 [00:03<00:00, 16.38it/s]
Epoch: 002, Train Loss: 6.6156156, Val macro-F1: 0.2113547
100%|██████████| 55/55 [00:03<00:00, 16.51it/s]
Epoch: 003, Train Loss: 5.0054732, Val macro-F1: 0.2476673
100%|██████████| 55/55 [00:03<00:00, 16.04it/s]
Epoch: 004, Train Loss: 2.7844433, Val macro-F1: 0.3196940
Improved val macro-F1 from 0.2849912307126517 to 0.3196940073302826 at epoch 4. Saving and logging model.
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war

100%|██████████| 55/55 [00:03<00:00, 16.68it/s]
Epoch: 007, Train Loss: 3.4265897, Val macro-F1: 0.3903089
Improved val macro-F1 from 0.3870166925465839 to 0.39030891881551855 at epoch 7. Saving and logging model.
100%|██████████| 55/55 [00:03<00:00, 16.75it/s]
Epoch: 008, Train Loss: 2.2330159, Val macro-F1: 0.4363459
Improved val macro-F1 from 0.39030891881551855 to 0.4363459352879648 at epoch 8. Saving and logging model.
100%|██████████| 55/55 [00:03<00:00, 16.67it/s]
Epoch: 009, Train Loss: 1.8138122, Val macro-F1: 0.4316325
100%|██████████| 55/55 [00:03<00:00, 16.50it/s]
Epoch: 010, Train Loss: 1.7993311, Val macro-F1: 0.4471370
Improved val macro-F1 from 0.4363459352879648 to 0.4471370236485341 at epoch 10. Saving and logging model.
Best val macro-F1: 0.4471370236485341
100%|██████████| 1/1 [02:20<00:00, 140.19s/it]
Tiny=True, group=family, train_ratio=1.0, model=gin, epochs=10, run time=48.3 seconds, # parameters=50629, layers=5, hidden_dims=64, learning_rate=0.0001, dropout=0, 


2023-03-06 05:11:44 Uploading - Uploading generated training model
2023-03-06 05:11:44 Completed - Training job completed
Training seconds: 398
Billable seconds: 398
CPU times: user 2.27 s, sys: 29.6 ms, total: 2.3 s
Wall time: 8min 22s


In [16]:
print(estimator.model_data)

s3://sagemaker-us-east-1-432088571089/malnet-graph-20230306050346/output/model.tar.gz


# backup

In [ ]:
import shutil
!pwd
src='./dm/tst'
dst='./dm/dst'
shutil.copytree(src, dst)